In [1]:
# %%capture
# %%script echo skipping
%load_ext autoreload
%load_ext dotenv
%dotenv
%pip install transformers evaluate
%pip install nltk absl-py rouge_score
%pip install bleu sacrebleu
%pip install sacremoses
%pip install scipy
%pip install sentencepiece
%pip install optimum auto-gptq
%pip install scikit-learn
%pip install einops
%pip install bitsandbytes
%pip install accelerate
!huggingface-cli login --token $HUGGING_FACE_TOKEN


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/izlobin/.cache/huggingface/token
Login successful


In [2]:
import os
from pprint import pprint

import evaluate
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from huggingface_hub import HfApi
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BartForCausalLM,
    BartModel,
    BartTokenizer,
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5TokenizerFast,
)
from utils.metric import calculate_scores

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pprint(f'Device: {device}')

'Device: cuda'


## Loading models

### Loading BART
* https://huggingface.co/facebook/bart-large

In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# model = BartModel.from_pretrained('facebook/bart-large')
model = BartForCausalLM.from_pretrained('facebook/bart-large', device_map=0)

print(model.config)

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state


BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_decoder": true,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_emb

### Loading t5
* https://huggingface.co/google-t5/t5-large

In [12]:
# model_name = "grammarly/coedit-large"
model_name = "google-t5/t5-large"
model_alias = model_name.replace("/", "_")

torch.cuda.empty_cache()

tokenizer = T5TokenizerFast.from_pretrained(model_name, model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map=0)

print(model.config)

T5Config {
  "_name_or_path": "google-t5/t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_lengt

### Loading coedit / flan-t5

In [17]:
# model_name = "grammarly/coedit-large"
model_name = "google/flan-t5-large"
model_alias = model_name.replace("/", "_")

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = T5ForConditionalGeneration.from_pretrained(model_name, device_map=0)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     # torch_dtype=torch.float16,
#     # revision="float16",
#     device_map="auto",
# )

In [16]:
# input_text = "Fix grammatical errors in this sentence: I goes to work every noning."
input_text = "Remove all grammatical errors from this text: For example, countries with a lot of deserts can fff their desert to increase their habitable land and using irrigation to provide clean water to the desert."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(
    **input_ids,
    do_sample=True,
    top_k=100,
    # top_p=0.9,
    # num_return_sequences=2,
    # eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)
# print(tokenizer.decode(outputs[0]))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# output = "For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert."
# reference = "For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert."

For example, countries with a lot of deserts can fff their desert to increase their habitable land and use irrigation to provide clean water to the desert.


### Loading Gemma

In [18]:
model_name = "google/gemma-2b-it"
# model_name = "google/gemma-7b-it"
# model_name = "google/gemma-7b"
model_alias = model_name.replace("/", "_")

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = T5ForConditionalGeneration.from_pretrained(model_name, device_map=0)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     # torch_dtype=torch.float16,
#     # revision="float16",
#     device_map="auto",
# )

print(model.config)

T5Config {
  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32128
}



### Loading Phi-2
* https://huggingface.co/microsoft/phi-2
* https://huggingface.co/TheBloke/phi-2-GPTQ

In [3]:
# model_name = 'microsoft/phi-2'
model_name = 'TheBloke/phi-2-GPTQ'
model_alias = model_name.replace('/', '_')

torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    trust_remote_code=True,
)

# print(f"model.config.eos_token_id: {model.config.eos_token_id}")
# eos_token_id = 50256 # https://huggingface.co/microsoft/phi-2/blob/main/config.json

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

print(f"tokenizer.eos_token: {tokenizer.eos_token}")
tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     # torch_dtype=torch.float16,
#     # revision="float16",
#     device_map="auto",
# )

print(model.config)

/home/izlobin/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer.eos_token: <|endoftext|>
PhiConfig {
  "_name_or_path": "TheBloke/phi-2-GPTQ",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "TheBloke/phi-2-GPTQ--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "TheBloke/phi-2-GPTQ--modeling_phi.PhiForCausalLM"
  },
  "embd_pdrop": 0.0,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "img_processor": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "phi-msft",
  "n_embd": 2560,
  "n_head": 32,
  "n_head_kv": null,
  "n_inner": null,
  "n_layer": 32,
  "n_positions": 2048,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.1,
    "dataset": null,
    "desc_act": true,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,

## Loading mixtral-8x7B
* https://huggingface.co/mistralai/Mixtral-8x7B-v0.1
* https://huggingface.co/TheBloke/Mixtral-8x7B-v0.1-GPTQ

Quantization
* https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996

In [7]:
# model_name = "mistralai/Mixtral-8x7B-v0.1"
model_name = 'TheBloke/Mixtral-8x7B-v0.1-GPTQ'
model_alias = model_name.replace('/', '_')

torch.cuda.empty_cache()

# from transformers import BitsAndBytesConfig
# quantization_config = BitsAndBytesConfig(load_in_4bit=4)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map=0,
#     # torch_dtype=torch.float16,
#     # load_in_4bit=True,
#     # quantization_config=quantization_config,
# )

# print(f"model.config.eos_token_id: {model.config.eos_token_id}")
# eos_token_id = 50256 # https://huggingface.co/microsoft/phi-2/blob/main/config.json

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    load_in_4bit=True,
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
)

# print(f"tokenizer.eos_token: {tokenizer.eos_token}")
# tokenizer.pad_token = tokenizer.eos_token


print(model.config)

# text = 'Hello my name is'
# inputs = tokenizer(text, return_tensors='pt')
# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


AttributeError: 'BitsAndBytesConfig' object has no attribute 'get_loading_attributes'

### Loading llama-2

In [3]:
# Loading llama-2

from transformers import LlamaForCausalLM, LlamaTokenizer
import sentencepiece as spm

# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "TheBloke/Llama-2-7B-GPTQ"
# model_name = "TheBloke/Nous-Hermes-Llama-2-7B-GPTQ"

model_alias = model_name.replace("/", "_")

tokenizer = LlamaTokenizer.from_pretrained(model_name, padding_side='left')
model = LlamaForCausalLM.from_pretrained(model_name, device_map=0)

print(model.config)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/izlobin/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its conten

LlamaConfig {
  "_name_or_path": "TheBloke/Llama-2-7B-GPTQ",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.01,
    "dataset": null,
    "desc_act": false,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,
    "max_input_length": null,
    "model_seqlen": null,
    "module_name_preceding_first_block": null,
    "modules_in_block_to_quantize": null,
    "pad_token_id": null,
    "quant_method": "gptq",
  

## Configuring memory (VRAM)

In [4]:
%reload_ext autoreload

from utils.monitoring import calculate_utilization, format_utilization_narrow, print_utilization

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total/trainable params: {total_params}/{total_trainable_params}')

utilization = calculate_utilization()
print(utilization)
utilization_str = format_utilization_narrow(utilization)
print(utilization_str)
print(
    f"total/used/cuda/res/ram(Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

actual_fraction = 0.95
available_memory = utilization['total_memory'] - utilization['memory_used']
recommended_fraction = available_memory / utilization['total_memory']
torch.cuda.set_per_process_memory_fraction(actual_fraction, 0)

print(
    f"Total/used/available memory (Gb): {utilization['total_memory']/1024**3:.2f}/"
    "{utilization['memory_used']/1024**3:.2f}/{available_memory/1024**3:.2f}"
)
print(f'Recommended/actual fraction: {recommended_fraction:.2f}/{actual_fraction:.2f}')

# torch.cuda.empty_cache()
# torch.empty(utilization['total_memory'] // 2, dtype=torch.int8, device='cuda')
# print_utilization()

Total/trainable params: 254084096/254084096
{'total_memory': 10736893952, 'memory_used': 3005644800, 'cuda_allocated': 1016336384, 'cuda_reserved': 1031798784, 'ram_usage': 13295087616}
{'total_memory': '10.00', 'memory_used': '2.80', 'cuda_allocated': '0.95', 'cuda_reserved': '0.96', 'ram_usage': '12.38'}
total/used/cuda/res/ram(Gb): 10.00/2.80/0.95/0.96/12.38
Total/used/available memory (Gb): 10.00/{utilization['memory_used']/1024**3:.2f}/{available_memory/1024**3:.2f}
Recommended/actual fraction: 0.72/0.95


## Test inference

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# reference = " Some even provided gateways, such as UFGATE, by which members could send / receive e-mail to and from the Internet via UUCP, and many FidoNet discussion groups were shared via gateway to Usenet."
input_text = "Fix grammatical errors in this sentence: I does work."
# input_ids = tokenizer(input_text, add_special_tokens=False, return_tensors="pt")
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Fix grammatical errors in this sentence: I does work.FfefefeDife Flowers Flowers-feeaeaeainaBeie Erin SweermMEferiainaFME Love LoveFER Flame Flame Erin Erin Swe Flame Erin Flame Flame Love Flame Erin Sweria Flame Flameriaria Flame Love Love Flameria Flame Erin Love Love Brandsriariariaca Flame Erin LOVE Love Flame Love Brands Flame Flame Flameilaria Flameermriaria Erin Flame Erin Wave Flame Flame LOVE Flame Erinila Flame Erin Flowersria Flameila Flame Flame Flowers Flame


In [17]:
# reference = " Some even provided gateways, such as UFGATE, by which members could send / receive e-mail to and from the Internet via UUCP, and many FidoNet discussion groups were shared via gateway to Usenet."
input_text = 'Fix grammatical errors in this sentence: I goes to work every noning.'
# input_ids = tokenizer(input_text, add_special_tokens=False, return_tensors="pt")
input_ids = tokenizer(input_text, return_tensors='pt')
input_ids = input_ids.to(device)

outputs = model.generate(
    **input_ids,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    # return_attention_mask=False,
    max_length=256,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Fix grammatical errors in this sentence: I goes to work every noning.
Answer: I go to work every day.

Exercise 3:
Add a conjunction to combine these two sentences: "I love to read." "I also enjoy playing sports."
Answer: "I love to read, and I also enjoy playing sports."

Exercise 4:
Complete the sentence by adding appropriate prepositions. "The cat is hiding __ the table."
Answer: "The cat is hiding under the table."

Exercise 5:
Rewrite the sentence by removing unnecessary words: "I went to the store, and then I also went to the bank."
Answer: "I went to the store and the bank."

I hope this comprehensive explanation, accompanied by real-world use cases and exercises, has piqued your interest and helped you understand the importance of grammar, conjunctions, and prepositions in the language arts. Remember, these elements are like the threads that hold our thoughts together, making communication clearer and more effective.

Stay curious and keep exploring the fascinating world of gra

### Datasets

In [5]:
# api = HfApi()
# coedit_info = api.dataset_info("grammarly/coedit")
# pprint(coedit_info)

grammarly_dataset = load_dataset("grammarly/coedit")
pprint(grammarly_dataset)

unique_categories = set(grammarly_dataset)
pprint(unique_categories)

unique_tasks = set(grammarly_dataset["train"]["task"])
pprint(unique_tasks)

DatasetDict({
    train: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 69071
    })
    validation: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})
{'validation', 'train'}
{'clarity', 'neutralize', 'coherence', 'gec', 'paraphrase', 'simplification'}


In [6]:
def get_samples(dataset, category="validation", task="gec", num_samples=1, seed=42):
    return dataset[category].shuffle(seed=seed).filter(lambda item: item["task"] == task).select(range(num_samples))

def print_samples(samples) -> None:
    for item in samples:
        pfx, src = item["src"].split(": ", 1)
        print(f"[{item['task']}] {pfx}")
        print(f"src: {src}")
        print(f"tgt: {item['tgt']}")


print_samples(get_samples(grammarly_dataset, num_samples=2))

# input_ids = tokenizer(item["task"], return_tensors="pt").input_ids.to(device)
# outputs = model.generate(input_ids, max_length=256)
# corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
# return {"processed": corrected}


[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
[gec] Fix grammaticality in this sentence
src: Dear friends, I hope you should correctly but I can gives you some opinion, I guess that is a good idea if you go to a small schools, under you can met a lot on people and there are more closed friend of course you cannot like that opcion if you like the biggest once, so in that ways you can go from the other school.
tgt: Dear friend, I hope you choose correctly but I can give you my opinion. I guess that it's a good idea if you go to a small school, because you can meet a lot of people and make more close friends of course you won't like that option if you like the bigger one, so in that case you should go to the other school.


### Metrics

In [7]:
rouge_metric = evaluate.load("rouge")
# glue_metric = evaluate.load("glue", "stsb")
sacreblue_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")
em_metric = evaluate.load("exact_match")

#### Rouge metric

In [7]:
samples = get_samples(grammarly_dataset, task="gec", num_samples=100)
pprint(samples)
print_samples([samples[0]])

score = rouge_metric.compute(
    predictions=samples['src'], references=samples['tgt']
)
pprint(score)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 100
})
[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
{'rouge1': 0.7150747966331101,
 'rouge2': 0.5167050375929942,
 'rougeL': 0.7005677840072502,
 'rougeLsum': 0.7007083564381789}


#### _GLUE metric_

In [8]:
samples = get_samples(grammarly_dataset, task="gec", num_samples=2)
pprint(object=samples)
print_samples([samples[0]])

src_input_ids = tokenizer(samples["src"][0], return_tensors="pt", padding=True).input_ids
tgt_input_ids = tokenizer(samples["tgt"][0], return_tensors="pt", padding=True).input_ids
pprint(src_input_ids[0])
pprint(tgt_input_ids[0])

# score = glue_metric.compute(predictions=src_input_ids[0], references=tgt_input_ids[0])
# score = glue_metric.compute(predictions=samples["src"], references=samples["tgt"])
# pprint(score)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 2
})
[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
tensor([14269, 19519,    16,    48,  7142,    10,   493,  6195,   388,    55,
            1])
tensor([ 493, 6195,    6,  388,   55,    1])


#### SacreBLEU metric

In [12]:
samples = get_samples(grammarly_dataset, task="gec", num_samples=100)
pprint(samples)
print_samples([samples[0]])

score = sacreblue_metric.compute(predictions=samples["src"], references=samples["tgt"])
pprint(score)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 100
})
[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
{'bp': 1.0,
 'counts': [3886, 2743, 1965, 1419],
 'precisions': [70.79613773000547,
                50.899981443681575,
                37.152580828133864,
                27.346309500867218],
 'ref_len': 5090,
 'score': 43.74251258938969,
 'sys_len': 5489,
 'totals': [5489, 5389, 5289, 5189]}


#### SARI metric

In [13]:
samples = get_samples(grammarly_dataset, task="gec", num_samples=100)
pprint(samples)
print_samples([samples[0]])

new_samples = samples.map(lambda item: {"tgts": [item["tgt"]]})
new_samples["tgts"][:5]

# sources=["About 95 species are currently accepted.","About 95 species are currently accepted."]
# predictions=["About 95 you now get in.","About 95 you now get in."]
# references=[["About 95 species are currently known.","About 95 species are now accepted.","95 species are now accepted."],["About 95 species are currently known.","About 95 species are now accepted.","95 species are now accepted."]]

score = sari_metric.compute(
  sources=new_samples['src'],
  predictions=new_samples['src'],
  references=new_samples['tgts']
)
pprint(score)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 100
})
[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
{'sari': 52.48853096503606}


#### Exact match (EM) metric

In [9]:
samples = get_samples(grammarly_dataset, task="gec", num_samples=100)
pprint(samples)
print_samples([samples[0]])

score = em_metric.compute(
    predictions=samples['tgt'], references=samples['tgt']
)
pprint(score)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 100
})
[gec] Fix grammar in this sentence
src: Be careful man!
tgt: Be careful, man!
{'exact_match': 1.0}


### Datasets

#### IteraTeR
* https://huggingface.co/datasets/wanyu/IteraTeR_v2
* https://huggingface.co/datasets/wanyu/IteraTeR_full_sent

In [8]:
# iterater_dataset = load_dataset("wanyu/IteraTeR_v2") # human in the loop
iterater_dataset = load_dataset("wanyu/IteraTeR_full_sent")
pprint(iterater_dataset)
iterater_validation_dataset = load_dataset("wanyu/IteraTeR_full_sent", split="validation")
pprint(iterater_validation_dataset)
# pprint(iterater_validation_dataset['validation'][0])

DatasetDict({
    train: Dataset({
        features: ['before_sent', 'before_sent_with_intent', 'after_sent', 'labels', 'confidence', 'doc_id', 'revision_depth'],
        num_rows: 157579
    })
    validation: Dataset({
        features: ['before_sent', 'before_sent_with_intent', 'after_sent', 'labels', 'confidence', 'doc_id', 'revision_depth'],
        num_rows: 19705
    })
    test: Dataset({
        features: ['before_sent', 'before_sent_with_intent', 'after_sent', 'labels', 'confidence', 'doc_id', 'revision_depth'],
        num_rows: 19703
    })
})
Dataset({
    features: ['before_sent', 'before_sent_with_intent', 'after_sent', 'labels', 'confidence', 'doc_id', 'revision_depth'],
    num_rows: 19705
})


In [9]:
verbolizers = {
    "gce": {
        "tokens": ["<fluency>"],
        "verbs": [
            "Fix grammar",
            "Fix grammar in this sentence",
            "Fix grammar in the sentence",
            "Fix grammar errors",
            "Fix grammatical errors",
            "Fix grammaticality",
            "Fix all grammatical errors",
            "Fix grammatical errors in this sentence",
            "Fix grammar errors in this sentence",
            "Fix grammatical mistakes in this sentence",
            "Fix grammaticality in this sentence",
            "Fix grammaticality of the sentence",
            "Fix disfluencies in the sentence",
            "Make the sentence grammatical",
            "Make the sentence fluent",
            "Fix errors in this text",
            "Update to remove grammar errors",
            "Remove all grammatical errors from this text",
            "Improve the grammar of this text",
            "Improve the grammaticality",
            "Improve the grammaticality of this text",
            "Improve the grammaticality of this sentence,",
            "Grammar improvements",
            "Remove grammar mistakes",
            "Remove grammatical mistakes",
            "Fix the grammar mistakes",
            "Fix grammatical mistakes",
        ],
    }
}

In [10]:
def substitute_verbolizer(text, verbolizer, count=[0]):
    verbs = verbolizers[verbolizer]["verbs"]

    verb = verbs[count[0]]
    tokens = verbolizers[verbolizer]["tokens"]
    replaced_text = text
    for t in tokens:
        replaced_text = text.replace(t, f"{verb}:")
        # pprint(f"> t: {t}, verb: {verb}, text: {text}, replaced_text: {replaced_text}")

    count[0] += 1
    if count[0] >= len(verbs):
        count[0] = 0

    return replaced_text


def get_iterater_samples(label, category="validation", num_samples=0, seed=42, confidence_threshold=0.9):
    filtered_samples = (
        iterater_dataset[category]
        .shuffle(seed=seed)
        .filter(lambda item: item["labels"] == label and float(item["confidence"]) >= confidence_threshold)
    )
    max_samples = len(filtered_samples)
    selected = max_samples if num_samples == 0 else num_samples
    print(f"max_samples: {max_samples}, selected: {selected}, num_samples: {num_samples}")
    samples = filtered_samples.select(range(selected))

    return samples.map(
        lambda item: {
            "task": substitute_verbolizer(item["before_sent_with_intent"], "gce"),
            "source": item["before_sent"],
            "reference": item["after_sent"],
            "references": [item["after_sent"]],
        },
        remove_columns=[
            "before_sent_with_intent",
            "before_sent",
            "after_sent",
            "labels",
            "confidence",
            "doc_id",
            "revision_depth",
        ],
    )


samples = get_iterater_samples(label="fluency", num_samples=5)
pprint(samples)
pprint(samples["task"][:2])

max_samples: 5078, selected: 5, num_samples: 5
Dataset({
    features: ['task', 'source', 'reference', 'references'],
    num_rows: 5
})
["Fix grammar:  We don't have enough good Open Source games -- it's a waste to "
 'pour all the resources we have into one. :) Wesnoth has dwarves with guns, '
 "World of Warcraft'' has gnomes and goblins with explosives and flying "
 'machines -- where do you, personally, define the limits of the fantasy '
 'genre?',
 'Fix grammar in this sentence:  In 2001, they successfully nominated Bohemian '
 'Hall, still a vibrant community center/beer garden started by Czech '
 'immigrants in Astoria, Queens, and the Casa Amadeo Music Store, the oldest, '
 'continuously occupied Latin music store in New York City,  as census sites '
 'to the National Register of Historic Places.']


### GPU processing

In [12]:
%%time

import time

batch_size = 10
total_samples = 100
samples = get_iterater_samples(label="fluency", num_samples=total_samples)
# samples = get_iterater_samples(label="fluency")
pprint(samples)

processed_samples = samples


from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, max_input_length=2400)

def model_process(batch, idx, **kwargs):
    num_samples = len(batch['task'])
    start_time = time.time()

    model = kwargs.get("model")
    tokenizer = kwargs.get("tokenizer")
    total_samples = kwargs.get("total_samples")

    input_ids = tokenizer(batch['task'], padding=True, return_tensors="pt").input_ids.to(device)
    # input_ids = tokenizer(batch['task'], return_tensors="pt").input_ids.to(device)
    # input_ids = tokenizer(item['task'], return_tensors="pt").input_ids
    # outputs = model.generate(input_ids, max_length=512)
    outputs = model.generate(input_ids, max_length=312)
    # print(f"outputs: {outputs}")
    processed = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    end_time = time.time()
    elapsed_time = end_time - start_time
    sps = num_samples / elapsed_time
    sps_str = f"{sps:.2f}"

    utilization = calculate_utilization()
    utilization_str = format_utilization_narrow(utilization)
    print(
        f"{idx[0]}-{idx[-1]}/{total_samples} | total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
        f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']} | "
        f"batch/sps: {num_samples}/{sps_str}"
    )

    # return {"processed": processed}
    return {"processed": processed}
    # return {"processed": processed, "utilization": utilization, "tps": tps}
    # return {"processed": processed, "utilization": utilization}


utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

start_time = time.time()

# processed_samples = samples.map(model_process, num_proc=torch.cuda.device_count())
processed_samples = samples.map(
    model_process,
    fn_kwargs={
        "model": model,
        "tokenizer": tokenizer,
        "total_samples": total_samples,
    },
    num_proc=1,
    batched=True,
    batch_size=batch_size,
    with_indices=True,
)

end_time = time.time()
elapsed_time = end_time - start_time
processed_sps = total_samples / elapsed_time
print(f"processed_sps: {processed_sps}")

pprint(processed_samples)
pprint(processed_samples['processed'][:2])

saved_samples = processed_samples.remove_columns(['references'])
flat_df = pd.DataFrame.from_records(saved_samples)
flat_df.to_json(f"samples/{model_alias}_frames.json", orient="records")

max_samples: 5078, selected: 100, num_samples: 100
Dataset({
    features: ['task', 'source', 'reference', 'references'],
    num_rows: 100
})
total/used/cuda/res/ram (Gb): 10.00/3.72/2.10/2.84/10.17


Map:  10%|█         | 10/100 [00:13<01:59,  1.33s/ examples]

0-9/100 | total/used/cuda/res/ram (Gb): 10.00/9.75/2.10/8.95/10.30 | batch/sps: 10/0.75


Map:  20%|██        | 20/100 [00:26<01:45,  1.32s/ examples]

10-19/100 | total/used/cuda/res/ram (Gb): 10.00/9.73/2.10/9.14/10.30 | batch/sps: 10/0.76


...

70-79/100 | total/used/cuda/res/ram (Gb): 10.00/9.84/2.10/9.37/10.22 | batch/sps: 10/0.74


Map:  90%|█████████ | 90/100 [01:49<00:12,  1.22s/ examples]

80-89/100 | total/used/cuda/res/ram (Gb): 10.00/9.94/2.10/9.37/10.23 | batch/sps: 10/0.71


Map: 100%|██████████| 100/100 [02:01<00:00,  1.21s/ examples]

90-99/100 | total/used/cuda/res/ram (Gb): 10.00/9.91/2.10/9.37/10.22 | batch/sps: 10/0.81
processed_sps: 0.8039060963211832
Dataset({
    features: ['task', 'source', 'reference', 'references', 'processed'],
    num_rows: 100
})
["Fix grammar:  We don't have enough good Open Source games -- it's a waste to "
 'pour all the resources we have into one. :) Wesnoth has dwarves with guns, '
 "World of Warcraft'' has gnomes and goblins with explosives and flying "
 'machines -- where do you, personally, define the limits of the fantasy '
 'genre?The.\n'
 'The study, published in the journal Nature, found that the bacteria in the '
 'gut of the mice were more diverse than those of the mice that were fed a '
 'standard diet.\n'
 'The researchers also found that the mice that were fed the high-fiber diet '
 'had a higher number of bacteria that were associated with the development of '
 'obesity.\n'
 'The researchers also found that the mice that were fed the high-fiber diet '
 'had a higher nu

In [13]:
scores = calculate_scores(processed_samples)
# pprint(scores)

score_paths = [
    "rouge.rouge1",
    # "rouge.rouge2",
    # "rouge.rougeL",
    # "rouge.rougeLsum",
    "sacreblue.score",
    "sari.sari",
    "em.exact_match",
]

base_frame = {
    "model": model_name,
    "total_samples": total_samples,
    "sps": processed_sps,
    "task": "fluency",
    "total_params": total_params,
}

normalized_scores = {}
for k, v in scores.items():
    for k2, v2 in v.items():
        if not isinstance(v2, list):
            # normalized_scores[f"score.{k}.{k2}"] = v2
            path = f"{k}.{k2}"
            if path in score_paths:
                normalized_scores[f"score.{k}.{k2}"] = v2
# pprint(normalized_scores)

flat_frame = base_frame.copy()
flat_frame.update(normalized_scores)
# pprint(frame)

all_flat_frames = []
if os.path.exists("results/all-flat-frames.csv"):
    all_flat_frames = pd.read_csv("results/all-flat-frames.csv").to_dict("records")

flat_df = pd.DataFrame.from_records([flat_frame])
# pprint(df)
# print(flat_df.head().to_markdown(index=True))
# flat_df.to_csv(f"results/{model_alias}_flat-frame.csv", index=False, float_format="%.2f")
flat_df.to_csv(f"results/{model_alias}_flat-frame.csv", index=False)

all_flat_frames.append(flat_frame)
all_flat_dfs = pd.DataFrame.from_records(all_flat_frames)
all_flat_dfs.to_csv(f"results/all-flat-frames.csv", index=False)


full_frame = base_frame.copy()
full_frame.update({"scores": scores})
# pprint(full_frame)
full_df = pd.DataFrame.from_records([full_frame])

pprint(full_df)
full_df.to_json(f"results/{model_alias}_full-frame.json", orient="records")
# full_df.to_json("results/frame.json", index=False)

# more_frames = [frame, frame, frame]
# more_df = pd.DataFrame.from_records(more_frames)
# more_df.to_csv("results/frames.csv", index=False)

                 model  total_samples       sps     task  total_params  \
0  TheBloke/phi-2-GPTQ            100  0.803906  fluency     262364160   

                                              scores  
0  {'rouge': {'rouge1': 0.3078833232553331, 'roug...  
